In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from priceanalytics.data import download_df_map, cache_df_map, load_cached_df_map 
from priceanalytics.plot import MultiPlot
from priceanalytics import indicators as I
from priceanalytics.backtest import Backtester

from datetime import datetime, date, time

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [63]:
tickers = ['AAPL', 'MSFT']
raw_tdfs = download_df_map(tickers, interval=1)
# cache_df_map(raw_tdfs, _dir="cache")


raw AAPL: 25802 datapoints from 2023-09-25 to 2023-11-11
raw MSFT: 22012 datapoints from 2023-09-25 to 2023-11-11


In [125]:
from priceanalytics.backtest import perc_ret
 
def test_and_strat(total, _len, sig_len):
    tpr = 1
    for n, df in enumerate(total):



        df = I.add_andean_osc(df, _len=_len, sig_len=sig_len)
        df = df.iloc[_len:]

        # with MultiPlot(2, size=(40,20)) as m:
        #     m.ohlc(df)
        #     m.next()
        #     plt.plot(np.array(df['andean_bull']), color='#00ff00')
        #     plt.plot(np.array(df['andean_bear']), color='#ff0000')
        #     plt.plot(np.array(df['andean_signal']), color='#000000')

        #print('\nSAMPLE #', n + 1)

        with Backtester(df, verbose=False) as b:
            for i, fr in b:
                if b.crossover("andean_bull", "andean_bear"):
                    b.buy()
    #                 print("buy @ $", b.at(i))

                elif b.crossover("andean_signal", "andean_bull"):
                    b.sell()
    #                 print("sell @ $", b.at(i))
    #             b.sell()

            incr = b.results(1)
            tpr *= (1 + incr)
            #print(f"returned {perc_ret(incr)}%")


    #print(f'\ntotal returned {perc_ret(tpr - 1)}%')
    return tpr - 1
    #         with MultiPlot(2, size=(40,20)) as m:
    #             m.ohlc(df)
    #             m.next()
    #             plt.plot(np.array(df['andean_bull']), color='#00ff00')
    #             plt.plot(np.array(df['andean_bear']), color='#ff0000')
    #             plt.plot(np.array(df['andean_signal']), color='#000000')

In [122]:
old_tdfs = load_cached_df_map(_dir="cache") # was @ interval = 5
old_dataset = old_tdfs['AAPL'] + old_tdfs['MSFT']

tickers = ['AAPL', 'MSFT', 'AMZN']
new_tdfs = download_df_map(tickers, interval=5)
new_dataset = []
for ticker, dfs in new_tdfs.items():
    new_dataset += dfs
    
dataset = old_dataset + new_dataset

datapts = sum([len(df) for df in dataset])
print("Loaded", len(dataset), "datasets containing", datapts, "datapoints")

raw AAPL: 6453 datapoints from 2023-09-25 to 2023-11-11
raw MSFT: 5881 datapoints from 2023-09-25 to 2023-11-11
raw AMZN: 6404 datapoints from 2023-09-25 to 2023-11-11
Loaded 128 datasets containing 23278 datapoints


In [129]:
tpr = test_and_strat(dataset, 10, 25)


In [148]:
# bulk test
results = {}

def dif_range(s, e):
    return range(s, e), s - e

test = 1
_len_range, ld = dif_range(4, 20)
sig_len_range, lr = dif_range(4, 40)
total = ld * lr

with open('and_results.json', 'w') as f:

    def save():
        f.write(json.dumps)


    for _len in _len_range:
        results[_len] = {}
        for sig_len in sig_len_range:
            tpr = test_and_strat(dataset, _len, sig_len)
            print(f"test={test}/{total} _len={_len} sig_len={sig_len} returns={perc_ret(tpr)}%")
            results[_len][sig_len] = tpr
            test += 1
            if test % 10 == 0:
                save()
    

test=1/576 _len=4 sig_len=4 returns=10.03%
test=2/576 _len=4 sig_len=5 returns=12.0%
test=3/576 _len=4 sig_len=6 returns=13.61%


KeyboardInterrupt: 

In [ ]:
print(results)